## Exercise 2: Using Large Language Models via APIs

In this exercise, you will learn how to interact with **large language models through APIs**, which is the most common way LLMs are used in research and industry. Instead of running a model locally, you will send requests to a remote model hosted by a provider and receive generated text as a response.

We will focus on the general principles of API-based LLM usage, which transfer across providers and models.

### Resources and starter code
- **API access**: All participants will be provided with an **API key** for the duration of the workshop.
- You will find **starter notebooks with example code** for using commercial LLMs in the provided **OpenAI notebook**.
- The notebook demonstrates:
  - API authentication
  - Sending prompts to a model
  - Controlling inference parameters
  - Processing model outputs programmatically

---

### Tasks

1. **Run the starter notebook**
   - Open the provided OpenAI notebook.
   - Verify that API access is working by running a simple prompt.

2. **Ask general questions**
   - Send a few questions to the model in English or Slovene.
   - Observe response quality, latency, and consistency.

3. **Machine translation**
   - Use the API to translate the following Slovenian sentence into English:
     > *Vlada Republike Slovenije je organ izvršilne oblasti, hkrati pa je tudi najvišji organ državne uprave.*

4. **Text summarization**
   - Provide a short news article as input.
   - Request a concise summary using the API.
   - Optionally constrain the output (e.g. maximum length, bullet points).

5. **Experiment with parameters**
   - Modify generation parameters such as:
     - `temperature`
     - `max_tokens`
     - `top_p`
   - Compare outputs across different configurations and tasks.

---

In [ ]:
!pip install openai==2.13.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 2.16.0
    Uninstalling openai-2.16.0:
      Successfully uninstalled openai-2.16.0


Exercise 2.2 Ask general questions:

In [2]:
from openai import OpenAI
from google.colab import userdata

API_KEY = userdata.get('API_KEY') # Save the API key in Secrets (in Colab - the key icon)

client = OpenAI(api_key=API_KEY)

response = client.chat.completions.create(
  model="gpt-4.1-mini",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Tell me which colour is the most popular in the world."}
  ],
  safety_identifier="T999"
)

message = response.choices[0].message.content
print(message)

The most popular color in the world is blue. Studies and surveys consistently show that blue is favored by the majority of people across different cultures and age groups. It is often associated with feelings of calmness, trust, and reliability.


Exercise 2.3 Machine translation:

In [3]:
from pydantic import BaseModel

class TranslationCandidates(BaseModel):
    original: str
    english: str

completion = client.beta.chat.completions.parse(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "You are a sentence translation system. Translate a Slovene sentence into English."},
        {"role": "user", "content": "Vlada Republike Slovenije je organ izvršilne oblasti, hkrati pa je tudi najvišji organ državne uprave."},
    ],
    response_format=TranslationCandidates,
)

event = completion.choices[0].message.parsed

In [4]:
import json

json_dict = json.loads(completion.choices[0].message.content)
print(json_dict)

{'original': 'Vlada Republike Slovenije je organ izvršilne oblasti, hkrati pa je tudi najvišji organ državne uprave.', 'english': 'The Government of the Republic of Slovenia is the executive authority and at the same time the highest body of the state administration.'}


Exercise 2.3 Text summarization:



In [6]:
# Text
text ="""
V letu, ki so ga v Kinodvoru zadovoljno končali z drugim najboljšim rezultatom po številu prodanih vstopnic v zgodovini mestnega kina,
je bil najbolj gledan film slovenska koprodukcija Fiume o morte!.
Na 1936 projekcijah so lani našteli 134.772 gledalcev v okviru 355 različnih programov. Skupni obisk v lanskem letu se uvršča takoj za
rekordno leto 2024, ko je Kinodvor v 16. letu delovanja kot mestni javni zavod dosegel mejnik 140.000 obiskovalcev. Obisk se jim zdi dober,
predvsem zato, ker sta bili dvorani zaradi prenove poleti nekaj časa zaprti. Želijo si širitve.
Fiume o morte! režiserja Igorja Bezinovića, ki je lani prejel več nagrad, med drugim evropsko filmsko nagrado za najboljši dokumentarec, si
je ogledalo 4061 gledalcev. Sicer pa je film vedno na sporedu.
Med peterico najbolj gledanih filmov mu sledijo Feničanska spletka Wesa Andersona (3818 gledalcev), Brutalist režiserja Bradyja Corbeta (3523 gledalcev), Konklave v režiji Edwarda Bergerja (2586 gledalcev) in Maria, ki ga je režiral Pablo Larrain (2385 gledalcev).
"""

# Example of response generation
response = client.chat.completions.create(
  model="gpt-4.1-mini",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Napiši povzetek v obliki alinej:\n\n" + text}
  ],
  safety_identifier="T999"
)

message = response.choices[0].message.content
print("Povzetek:", message)


Povzetek: - Kinodvor je v lanskem letu dosegel drugi najboljši rezultat po številu prodanih vstopnic v zgodovini mestnega kina.
- Najbolj gledan film je bila slovenska koprodukcija "Fiume o morte!" režiserja Igorja Bezinovića.
- V letu 2023 je Kinodvor izvedel 1936 projekcij z 134.772 gledalci v okviru 355 različnih programov.
- Skupni obisk se uvršča takoj za rekordno leto 2024, ko je Kinodvor dosegel 140.000 obiskovalcev.
- Dvorani sta bili poleti zaradi prenove nekaj časa zaprti, kar je vplivalo na obisk; kino si želi širitve.
- Film "Fiume o morte!" je prejel več nagrad, med drugim evropsko filmsko nagrado za najboljši dokumentarec, in si ga je ogledalo 4061 gledalcev.
- Med petimi najbolj gledanimi filmi sledijo:
  - "Feničanska spletka" Wesa Andersona (3818 gledalcev)
  - "Brutalist" Bradyja Corbeta (3523 gledalcev)
  - "Konklava" Edwarda Bergerja (2586 gledalcev)
  - "Maria" Pabla Larraina (2385 gledalcev).


Exercise 2.4 Experiment with parameters:

In [14]:

# Example of response generation
response = client.chat.completions.create(
  model="gpt-4.1-mini",
  messages=[
    {"role": "system", "content": "You are an expert writer in Slovene."},
    {"role": "user", "content": "Na kratko opiši kravo."}
  ],
  safety_identifier="T999",
  temperature=0.1
)

message = response.choices[0].message.content
print("Povzetek:", message)

Povzetek: Krava je domača prežvekovalka, ki jo gojimo predvsem zaradi mleka, mesa in usnja. Ima močno telo, štiri želodčne predele in značilno rjavo, belo ali črno-rjavo barvo dlake. Krave so družabne živali, ki živijo v čredah.


In [16]:
# Example of response generation
response = client.chat.completions.create(
  model="gpt-4.1-mini",
  messages=[
    {"role": "system", "content": "You are an expert writer in Slovene."},
    {"role": "user", "content": "Na kratko opiši kravo."}
  ],
  safety_identifier="T999",
  temperature=1
)

message = response.choices[0].message.content
print("Povzetek:", message)

Povzetek: Krava je domača prežvekovalka iz družine goved, ki jo ljudje vzrejajo predvsem zaradi mleka, mesa in usnja. Ima močno telo, kratek rep in velike oči. Krave so znane po svoji mirni naravi in so pomemben del kmetijstva po vsem svetu.
